In [7]:
import csv
import pprint

import pandas as pd
from pyserini.index import IndexReader

In [8]:
index_reader = IndexReader("../indexes/msmarco-doc/lucene-index-msmarco")



In [9]:
rundict = {}
with open('../data/run.msmarco-doc.test.bm25.txt') as runfile:
    reader = csv.reader(runfile, delimiter=' ')

    count = 0
    for [qid, _, docid, _, _, _] in reader:
        if qid not in rundict:
            rundict[qid] = []
        rundict[qid].append(docid)

In [10]:
rundict = {k:v[:100] for k,v in rundict.items()}

{'57': ['D410337',
  'D1159768',
  'D3489369',
  'D1335737',
  'D736559',
  'D3112315',
  'D2203564',
  'D275593',
  'D610619',
  'D665223',
  'D429416',
  'D2789782',
  'D1791019',
  'D472028',
  'D320151',
  'D2322243',
  'D1930842',
  'D101188',
  'D608870',
  'D2439948',
  'D2244829',
  'D2739664',
  'D932648',
  'D1234475',
  'D144976',
  'D101189',
  'D209110',
  'D2655384',
  'D3469662',
  'D1997586',
  'D1621880',
  'D1305636',
  'D198573',
  'D1403771',
  'D2233994',
  'D66645',
  'D320147',
  'D93434',
  'D2266416',
  'D2447775',
  'D584974',
  'D1873856',
  'D632443',
  'D2915048',
  'D2432453',
  'D647001',
  'D321565',
  'D2141433',
  'D2955290',
  'D3210752',
  'D2078037',
  'D1336396',
  'D2690912',
  'D289176',
  'D924204',
  'D1077367',
  'D3417935',
  'D2990673',
  'D2484511',
  'D53760',
  'D3018944',
  'D659963',
  'D323571',
  'D458541',
  'D1475385',
  'D942411',
  'D652017',
  'D1055177',
  'D2083273',
  'D2594260',
  'D3533009',
  'D2970543',
  'D2166772',
  'D1

In [11]:
test_qrels = {}
with open('../data/msmarco-doctest-qrels.tsv') as qrels:
    reader = csv.reader(qrels, delimiter=' ')

    for [qid, _, docid, rel] in reader:
        if qid not in test_qrels:
            test_qrels[qid] = {}
        test_qrels[qid][docid] = rel

In [12]:
qdict = {}
with open('../data/msmarco-doctest-queries.tsv') as qfile:
    reader = csv.reader(qfile, delimiter='\t')
    for [qid, q] in reader:
        qdict[qid] = q

In [13]:
metric_dict = {}


with open('../data/bm25_metrics.txt') as mf:
    reader = csv.reader(mf, delimiter='\t')
    for [metric, id, value] in reader:
        if id not in metric_dict:
            metric_dict[id] = {}
        metric_dict[id][metric] = value
metric_list = [[k] + list(v.values()) for k, v in metric_dict.items()]



In [14]:
top100dict = {}

with open('../data/msmarco-doctest-top100.tsv') as tf:
    reader = csv.reader(tf, delimiter=' ')
    for [qid, _, docid, _, _, _] in reader:
        if qid not in top100dict:
            top100dict[qid] = []
        top100dict[qid].append(docid)
# top100dict

In [15]:
def df_from_records(records):

    df = pd.DataFrame.from_records(records, columns=['qid','map', 'mrr', 'ndcg'])
    return df



In [16]:
df = df_from_records(metric_list)

In [17]:
df.describe()

,qid,map,mrr,ndcg
count,44,44,44,44
unique,44,44,7,43
top,1063750,0.3381,1.0000,0.0000
freq,1,1,30,2


In [18]:
df.sort_values(by=['map'], ascending=False).index

Int64Index([38, 42, 10, 12, 21, 34, 18, 17, 15, 22, 13, 29, 16, 27, 25, 26, 24,
            43, 41, 19, 36, 35, 28,  8, 20, 39, 40, 23,  5,  9, 37, 14,  1, 32,
             7,  6,  3, 11,  2, 33, 31,  4, 30,  0],
           dtype='int64')

In [19]:
df.sort_values(by=['mrr'], ascending=False).index

Int64Index([16, 32, 35, 29, 28, 36, 26, 25, 24, 23, 37, 21, 38, 19, 18, 17, 34,
            15, 14, 13, 12, 11, 10,  9,  8, 40,  5,  4, 42,  2, 43, 39, 22, 31,
            20,  6,  3,  1,  7, 41, 33, 27, 30,  0],
           dtype='int64')

In [20]:
df.sort_values(by=['ndcg'], ascending=False).index


Int64Index([38, 42, 12, 28, 15, 10, 17, 18, 13, 39,  9, 23, 37,  8, 16, 40, 21,
            29, 36, 25, 26, 19, 11, 34, 22, 20, 43, 27, 24,  5, 35, 32,  3, 14,
             2,  4, 41,  6, 31,  7, 33,  1, 30,  0],
           dtype='int64')

In [119]:
qid = df.iloc[0]['qid']

In [120]:
doclist = rundict[qid]

In [121]:
pp = pprint.PrettyPrinter(indent=4)

In [122]:
pp.pprint(qdict[qid])

'who is robert gray'


In [126]:
relrel = 0
reldocs_retr = []
for docid in doclist:
    rel = test_qrels[qid].get(docid,0)
    if int(rel) != 0:
        relrel+=1
        reldocs_retr.append(docid)
pp.pprint(f"Retr rels for {qid}: {relrel}")



'Retr rels for 1037798: 3'


In [127]:
relrel = 0
reldocs_og = []
for docid, label in test_qrels[qid].items():
    if int(label) != 0:
        relrel += 1
        reldocs_og.append(docid)
print(f"OG rels for {qid}: {relrel}")

OG rels for 1037798: 44


In [128]:
unretr = [docid for docid in reldocs_og if not docid in reldocs_retr]
print(unretr)

for unretrv in unretr:
    doc_raw = index_reader.doc_raw(unretrv)
    pp.pprint(f"rank: {reldocs_og.index(unretrv) + 1}, docid: {unretrv}, {doc_raw[:100]}")

['D1229426', 'D1397858', 'D1408563', 'D1433811', 'D1433813', 'D146618', 'D1476613', 'D1511498', 'D1630835', 'D1656125', 'D1669646', 'D1736139', 'D1805868', 'D1840248', 'D1884178', 'D1908406', 'D2340924', 'D2390607', 'D2531956', 'D2589783', 'D2603229', 'D2603230', 'D2663779', 'D2832775', 'D2893134', 'D292289', 'D292295', 'D3090861', 'D312949', 'D3320126', 'D3336258', 'D3338700', 'D3517959', 'D402272', 'D449680', 'D490083', 'D590671', 'D607139', 'D625738', 'D78009', 'D88119']
('rank: 1, docid: D1229426, <TEXT>\n'
 'https://en.wikipedia.org/wiki/Geography_of_Washington\n'
 'Washington (state)\n'
 'Washington (state)Fr')
('rank: 2, docid: D1397858, <TEXT>\n'
 'https://en.wikipedia.org/wiki/Washougal,_Washington\n'
 'Washougal, Washington\n'
 'Washougal, Washingt')
('rank: 3, docid: D1408563, <TEXT>\n'
 'https://en.wikipedia.org/wiki/History_of_Oregon\n'
 'History of Oregon\n'
 'From Wikipedia, the free en')
('rank: 4, docid: D1433811, <TEXT>\n'
 'https://en.wikipedia.org/wiki/Talk:Lewis_an

In [81]:
[docid for docid in reldocs_retr if not docid in reldocs_og]

[]

In [129]:
qrel_docs = [k for k,v in test_qrels[qid].items() if int(v) != 0]


In [130]:
qrels_unretr = [docid for docid in qrel_docs if not docid in reldocs_retr]

In [133]:
for unretrv in qrels_unretr:
    doc_raw = index_reader.doc_raw(unretrv)
    pp.pprint(f"docid: {unretrv}, {doc_raw[:1000]}")


('docid: D1229426, <TEXT>\n'
 'https://en.wikipedia.org/wiki/Geography_of_Washington\n'
 'Washington (state)\n'
 'Washington (state)From Wikipedia, the free encyclopedia (Redirected from '
 'Geography of Washington)navigation search"Washington state" and "State of '
 'Washington" redirect here.\n'
 'For other uses, see Washington (disambiguation).\n'
 'State of Washington\n'
 'Flag Seal\n'
 'Nickname (s): "The Evergreen State"  (unofficial)  [1]Motto (s): Al-ki or '
 'Alki, "bye and bye" in Chinook Jargon  (unofficial)  [1]State song (s): " '
 'Washington, My Home "Official language None  (de jure) English  (de '
 'facto)Demonym Washingtonian\n'
 'Capital Olympia\n'
 'Largest city Seattle\n'
 'Largest metro Metro Seattle\n'
 'Area Ranked 18th• Total 71,362 sq mi  (184,827 km 2)• Width 360 miles (580 '
 'km)• Length 240 miles (400 km)• % water 6.6• Latitude 45° \u200a33′ N to 49° '
 'N• Longitude 116° \u200a55′ W to 124° \u200a46′ WPopulation Ranked 13th• '
 'Total 7,405,743 (2017 est.)

In [143]:
analys = index_reader.analyze('who is robert gray')


In [152]:
for unretrv in qrels_unretr:
    termcount = 0
    vec = index_reader.get_document_vector(unretrv)
    raw_len = len(index_reader.doc_raw(unretrv))

    for term in analys:
        termcount += vec.get(term,0)
    tot = sum([vv for vv in vec.values() if vv is not None])
    pp.pprint(f"{unretrv}: {termcount}/{tot} -> {round(termcount/tot * 100,2)}%, {round(termcount/raw_len * 100,2)}%")

'D1229426: 12/11028 -> 0.11%, 0.01%'
'D1397858: 13/1947 -> 0.67%, 0.08%'
'D1408563: 6/3427 -> 0.18%, 0.02%'
'D1433811: 13/5993 -> 0.22%, 0.03%'
'D1433813: 11/2337 -> 0.47%, 0.05%'
'D146618: 8/1931 -> 0.41%, 0.05%'
'D1476613: 9/3030 -> 0.3%, 0.03%'
'D1511498: 5/1663 -> 0.3%, 0.04%'
'D1630835: 12/11023 -> 0.11%, 0.01%'
'D1656125: 6/319 -> 1.88%, 0.22%'
'D1669646: 4/82 -> 4.88%, 0.56%'
'D1736139: 8/845 -> 0.95%, 0.11%'
'D1805868: 28/7422 -> 0.38%, 0.05%'
'D1840248: 12/11027 -> 0.11%, 0.01%'
'D1884178: 18/2566 -> 0.7%, 0.08%'
'D1908406: 3/194 -> 1.55%, 0.17%'
'D2340924: 10/1831 -> 0.55%, 0.06%'
'D2390607: 4/640 -> 0.62%, 0.07%'
'D2531956: 5/1121 -> 0.45%, 0.05%'
'D2589783: 5/1529 -> 0.33%, 0.04%'
'D2603229: 9/3035 -> 0.3%, 0.03%'
'D2603230: 5/697 -> 0.72%, 0.08%'
'D2663779: 5/304 -> 1.64%, 0.18%'
'D2832775: 12/11053 -> 0.11%, 0.01%'
'D2893134: 6/2521 -> 0.24%, 0.03%'
'D292289: 8/6065 -> 0.13%, 0.01%'
'D292295: 3/385 -> 0.78%, 0.09%'
'D3090861: 22/3631 -> 0.61%, 0.07%'
'D312949: 28/7416 -> 

In [ ]:
    pp.pprint(len(index_reader.doc_raw(qrels_unretr[0])))
pp.pprint(index_reader.doc_raw(qrels_unretr[0]))
# pp.pprint(len(index_reader.doc_raw(qrels_unretr[1])))
# pp.pprint(len(index_reader.doc_raw(qrels_unretr[2])))

['who', 'robert', 'grai']

In [139]:
pp.pprint(index_reader.get_document_vector(qrels_unretr[0])['who'])
pp.pprint(index_reader.get_document_vector(qrels_unretr[0])['robert'])
pp.pprint(index_reader.get_document_vector(qrels_unretr[0])['grai'])

5
1
6


In [113]:
for i in range(0,10):
    docid = doclist[i]
    rel = test_qrels[qid][docid]
    if int(rel) != 0:
        continue
    pp.pprint(f"{i+1} {docid}, label: {test_qrels[qid][docid]}")



    raw_len = len(index_reader.doc(docid).raw())

    pp.pprint(f"length: {raw_len}")

'8 D1222360, label: 0'
'length: 15118'


In [116]:
pp.pprint(index_reader.doc_raw('D1222360')[:10])
pp.pprint(index_reader.get_document_vector('D1222360')['salvat'])
pp.pprint(index_reader.get_document_vector('D1222360')['armi'])
pp.pprint(index_reader.get_document_vector('D1222360')['found'])
pp.pprint(index_reader.get_document_vector('D1222360')['when'])
pp.pprint(len(index_reader.doc_raw('D1222360')))


'<TEXT>\nhtt'
24
52
2
4
15118


In [118]:
pp.pprint(index_reader.get_term_counts('salvation army'))
pp.pprint(index_reader.get_term_counts('salvation'))
pp.pprint(index_reader.get_term_counts('army'))

(1811, None)
(11566, 28688)
(104679, 443034)


In [47]:
for i in range(0,10):
    docid = top100dict[qid][i]
    rel = test_qrels[qid][docid]
    if int(rel) == 0:
        continue
    pp.pprint(f"{i+1} {docid}, label: {test_qrels[qid][docid]}")



    raw_len = len(index_reader.doc(docid).raw())

    pp.pprint(f"length: {raw_len}")


'1 D3542887, label: 3'
'length: 3179'
'2 D2740107, label: 3'
'length: 74447'
'3 D3178882, label: 3'
'length: 72919'
'4 D3542888, label: 3'
'length: 35507'
'5 D1496110, label: 3'
'length: 26229'
'6 D1752868, label: 3'
'length: 8212'
'7 D1921574, label: 1'
'length: 5395'
'9 D1496113, label: 3'
'length: 22315'


In [48]:
index_reader.get_term_counts('theraderm', analyzer=None)

(6, 30)

In [73]:
index_reader.stats()

{'total_terms': 2748636047,
 'documents': 3213835,
 'non_empty_documents': 3213835,
 'unique_terms': -1}